# Debug notebook
This notebook is only used to troubleshoot the S2 ARD workflow processor.

## S2 ARD workflow

In [1]:
import json
from utils.prepLS import prepareLS

def process_scene(json_data):
    loaded_json = json.loads(json_data)
    #for x in loaded_json:
    #    print("%s: %s" % (x, loaded_json[x]))
    prepareLS(**loaded_json)

no gsutils installed
sentinelsat not installed


## S2 ARD job queue processor

In [2]:
import rediswq

host="redis-master4"
# Uncomment next two lines if you do not have Kube-DNS working.
# import os
# host = os.getenv("REDIS_SERVICE_HOST")

q = rediswq.RedisWQ(name="jobLS", host=host)
print("Worker with sessionID: " +  q.sessionID())
print("Initial queue state: empty=" + str(q.empty()))

Worker with sessionID: bbe05454-1b99-4517-9b00-bf7544509c59
Initial queue state: empty=False


In [3]:
while not q.empty():
    item = q.lease(lease_secs=1800, block=True, timeout=600)
    if item is not None:
        itemstr = item.decode("utf=8")
        print("Working on " + itemstr)
        #time.sleep(10) # Put your actual work here instead of sleep.
        process_scene(itemstr)
        q.complete(item)
    else:
        print("Waiting for work")

Working on {"ls_url": "https://edclpdsftp.cr.usgs.gov/orders/espa-Sarah.Cheesbrough@sa.catapult.org.uk-10252019-080031-821/LC080750722015080501T1-SC20191025150256.tar.gz", "s3_bucket": "public-eo-data", "s3_dir": "fiji/landsat_8_t/"}
scene: LC08_L1TP_075072_20150805
untar: /tmp/data/intermediate/LC08_L1TP_075072_20150805_tmp/LC08_L1TP_075072_20150805_untar/
cog_dir/tmp/data/intermediate/LC08_L1TP_075072_20150805_tmp/LC08_L1TP_075072_20150805/
Extracting tar.gz: /tmp/data/intermediate/LC08_L1TP_075072_20150805_tmp/LC080750722015080501T1-SC20191025150256.tar.gz
/tmp/data/intermediate/LC08_L1TP_075072_20150805_tmp/LC08_L1TP_075072_20150805_untar/LC08_L1TP_075072_20150805_20170406_01_T1_bt_band10.tif
/tmp/data/intermediate/LC08_L1TP_075072_20150805_tmp/LC08_L1TP_075072_20150805_untar/LC08_L1TP_075072_20150805_20170406_01_T1_bt_band11.tif
/tmp/data/intermediate/LC08_L1TP_075072_20150805_tmp/LC08_L1TP_075072_20150805_untar/LC08_L1TP_075072_20150805_20170406_01_T1_pixel_qa.tif
/tmp/data/inter

KeyboardInterrupt: 

In [ ]:
print("Queue empty!")